In [ ]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# datasets

In [ ]:
# path = join_path(DATA_DIR, DATASET)
# !mkdir -p {path}

In [ ]:
# for name in DATASETS:
#     paths = (
#         join_path(CORUS_DATA_DIR, _)
#         for _ in CORUS_FILES[name]
#     )
#     records = (
#         record
#         for path in paths
#         for record in load_dataset(path)
#     )
#     records = log_progress(records, desc=name)
#     records = sample(records, 1000)

#     path = join_path(DATA_DIR, DATASET, name + JL + GZ)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
datasets = {}
for name in DATASETS:
    path = join_path(DATA_DIR, DATASET, name + JL + GZ)
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    datasets[name] = list(from_jsons(items, Markup))

# models

In [ ]:
# for name in MODELS:
#     path = join_path(DATA_DIR, name)
#     !mkdir -p {path}

## cpu

In [ ]:
# docker = docker_client()

In [ ]:
# model_name = SPACY
# model = MODELS[model_name]()
# model.start(docker)
# model.wait()

In [ ]:
# for dataset_name in DATASETS:
#     records = model.map(_.words for _ in datasets[dataset_name])
#     records = log_progress(records, desc=dataset_name)

#     path = join_path(DATA_DIR, model_name, dataset_name + JL + GZ)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
# model.stop(docker)

## gpu

In [ ]:
# !vast search offers | grep '1 x  GTX 1080 Ti'

In [ ]:
# model = DeeppavlovModel()
# model = DeeppavlovBERTModel()
# model = SlovnetBERTModel()
model = StanzaModel()

In [ ]:
# !vast create instance 498741 --image {model.image} --disk 30

In [ ]:
# !vast show instances

In [ ]:
# !ssh ssh4.vast.ai -p 20861 -l root -Nf -L {model.port}:localhost:{model.container_port}

In [ ]:
# for dataset_name in DATASETS:
#     records = datasets[dataset_name]
#     records = log_progress(records, desc=dataset_name)
#     records = model.map(_.words for _ in records)

#     path = join_path(DATA_DIR, model.name, dataset_name + JL + GZ)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
# !vast destroy instance 500861

# score

In [ ]:
dataset_models = {}
for dataset in DATASETS:
    for model in MODELS:
        path = join_path(DATA_DIR, model, dataset + JL + GZ)
        lines = load_gz_lines(path)
        items = parse_jl(lines)
        dataset_models[dataset, model] = list(from_jsons(items, Markup))

In [ ]:
scores = {}
for dataset, model in log_progress(dataset_models):
    preds = dataset_models[dataset, model]
    targets = datasets[dataset]
    scores[dataset, model] = score_markups(preds, targets)

# report

In [ ]:
scores_table = scores_report_table(scores, DATASETS, MODELS)
html = format_scores_report(scores_table)
patch_readme(MORPH1, html, README)
patch_readme(MORPH1, html, SLOVNET_README)
HTML(html)

In [ ]:
BENCH = [
    Bench(
        DEEPPAVLOV,
        init=4,
        disk=32 * MB,
        ram=10 * GB,
        speed=90,
        device=GPU
    ),
    Bench(
        DEEPPAVLOV_BERT,
        init=20,
        disk=(706 + 687) * MB,  # BERT + model
        ram=8.5 * GB,
        speed=85,
        device=GPU
    ),
    Bench(
        SLOVNET_BERT,
        init=5,
        disk=475 * MB,
        ram=8087 * MB,
        speed=285,
        device=GPU
    ),
    Bench(
        SLOVNET,
        init=1,
        disk=27 * MB,
        ram=115 * MB,
        speed=532,
    ),

    Bench(
        UDPIPE,
        init=6.91,
        disk=45 * MB,
        ram=242 * MB,
        speed=56.2,
    ),
    Bench(
        SPACY,
        init=8,
        disk=140 * MB,
        ram=579 * MB,
        speed=50,
    ),
    Bench(
        MARU,
        init=15.8,
        disk=44 * MB,
        ram=370 * MB,
        speed=36.4,
    ),
    Bench(
        RNNMORPH,
        init=8.73,
        disk=10 * MB,
        ram=289 * MB,
        speed=16.6,
    ),
    Bench(
        RUPOSTAGGER,
        init=4.81,
        disk=2.7 * MB,
        ram=118 * MB,
        speed=48,
    ),
    Bench(
        STANZA,
        init=2,
        disk=591 * MB,
        ram=393 * MB,
        speed=92,
    ),
]

bench_table = bench_report_table(BENCH, MODELS)
html = format_bench_report(bench_table)
patch_readme(MORPH2, html, README)
patch_readme(MORPH2, html, SLOVNET_README)
HTML(html)